In [2]:
import pandas as pd
import os
from glob import glob
import reports
pd.options.display.float_format = '{:.2f}'.format

In [4]:
no_pt_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/prev_KeyBERT/finetune_{}/source=*_post=None_target=*/classification_report.csv'
pt_raw_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/prev_KeyBERT/finetune_{}/source=*_post=*-raw_target=*/classification_report.csv'
pt_random_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/prev_KeyBERT/finetune_{}/source=*_post=*-random_target=*/classification_report.csv'
pt_keyword_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=*-ds-keyword_target=*/classification_report.csv'

In [5]:
def get_average_classification_report(reports):
    for i, report in enumerate(reports):
        report.set_index(report.columns[0], inplace=True)
        if i == 0: 
            report_avg = report
        else: 
            report_avg = report_avg.add(report)
    report_avg = report_avg / len(reports)
    return report_avg

def show_result(filepaths_format, kfold_num=1):
    kfold_reports = []
    for i in range(0, kfold_num):
        filepaths = glob(filepaths_format.format(i))

        records = []
        for filepath in filepaths:
            source, post, target = os.path.basename(os.path.dirname(filepath)).split('_')
            source, target = source.replace('source=',''), target.replace('target=', '')

            df = pd.read_csv(filepath)
            df.set_index('Unnamed: 0', inplace=True)
            acc = 100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]

            records.append(('{}->{}'.format(source, target), float(acc)))

        result_df = pd.DataFrame(records, columns = ['source->target', '{}_accuracy'])
        kfold_reports.append(result_df)

    result_df = get_average_classification_report(kfold_reports)
    result_df.loc['Average'] = result_df.mean()
    return result_df

Logistic Regression 이용하여 domain-specific keywords 추출 후 UNK 처리하여 post-train
* Raw 소스, Raw target 테스트
* No k-fold

In [6]:
no_pt_df = show_result(no_pt_filepaths_format)
no_pt_df.columns = ['no_pt']

pt_raw = show_result(pt_raw_filepaths_format)
pt_raw.columns = ['RAW']

pt_random = show_result(pt_random_filepaths_format)
pt_random.columns = ['RANDOM']

pt_keyword = show_result(pt_keyword_filepaths_format)
pt_keyword.columns = ['domain-specific-KEYWORD']

result = pd.concat([no_pt_df, pt_raw, pt_random, pt_keyword], axis=1)
result

,no_pt,RAW,RANDOM,domain-specific-KEYWORD
books->dvd,88.60,90.30,90.10,89.25
books->electronics,86.55,90.00,89.40,88.80
electronics->books,84.85,88.85,88.70,86.95
electronics->dvd,83.45,89.50,88.70,87.55
electronics->kitchen,87.05,92.70,92.60,92.30
kitchen->books,85.60,89.70,89.35,82.15
kitchen->dvd,84.95,88.30,88.75,83.30
kitchen->electronics,88.90,92.45,92.20,92.15
books->kitchen,88.60,91.35,91.40,91.25
dvd->kitchen,85.00,90.95,90.25,89.75
